In [1]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.pipeline import make_pipeline
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from sklearn.preprocessing import StandardScaler

In [2]:
#path1 = "https://raw.githubusercontent.com/jaarciah/Proyecto-Integrador-2020-periodo-2/master/Data/Preprocessing/hurto_motos_categorias_ingCaracteristicas.csv"
path2 = "https://raw.githubusercontent.com/jaarciah/Proyecto-Integrador-2020-periodo-2/master/Data/Modeling/hurto_preprocesado_C.csv"

#hurtos_moto = pd.read_csv(path1, sep=";", low_memory=False)
hurto_preprocesado = pd.read_csv(path2, sep=",", low_memory=False)

In [3]:
hurto_preprocesado.modalidad.value_counts()

0    33100
1    25455
Name: modalidad, dtype: int64

In [4]:
hurto_preprocesado.head()

,latitud,longitud,estado_civil,medio_transporte,modalidad,nombre_barrio,codigo_comuna,lugar,sede_receptora,modelo,...,anho,mes,dia,dia_semana,hora,festivos,quincena,week_number,ferias_fiestas,franja_horaria
0,6.268040,-75.549892,0,0,0,0,0,0,0,0,...,2017,1,1,6,23,1,1,52,0,2
1,6.268129,-75.557882,0,0,0,4,3,2,3,4,...,2017,1,1,6,16,1,1,52,0,1
2,6.268399,-75.549576,0,0,0,0,0,0,0,4,...,2017,1,1,6,10,1,1,52,0,0
3,6.263576,-75.558601,3,0,0,4,3,0,3,4,...,2017,1,3,1,8,0,0,1,0,0
4,6.263299,-75.556534,0,1,1,4,3,0,3,0,...,2017,1,7,5,3,0,0,1,0,2


In [5]:
hurto_preprocesado.columns

Index(['latitud', 'longitud', 'estado_civil', 'medio_transporte', 'modalidad',
       'nombre_barrio', 'codigo_comuna', 'lugar', 'sede_receptora', 'modelo',
       'fecha', 'anho', 'mes', 'dia', 'dia_semana', 'hora', 'festivos',
       'quincena', 'week_number', 'ferias_fiestas', 'franja_horaria'],
      dtype='object')

In [6]:
# variables escogidas despues del proceso de ingenieria de caracteristica.
variables = ["latitud","longitud","estado_civil","medio_transporte","modalidad","nombre_barrio","codigo_comuna","lugar","sede_receptora","modelo", "fecha"]

hurto_preprocesado_new = hurto_preprocesado[variables]

# Proceso de cross-validation.

In [7]:
y = hurto_preprocesado_new.modalidad
x = hurto_preprocesado_new.drop(['modalidad'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=8)

# 7. Ensemble methods

## 1. Bagging Classifier.
https://scikit-learn.org/stable/modules/ensemble.html#bagging-meta-estimator

### a. Decision Tree.

In [9]:
modelbaggingDTre = make_pipeline(BaggingClassifier(DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,criterion='entropy', max_depth=9,max_features=None, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=4, min_weight_fraction_leaf=0.0, presort='deprecated', random_state=None, splitter='best')))
modelbaggingDTre.fit(X_train,
          y_train)
ypred = modelbaggingDTre.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      6657
           1       0.86      0.89      0.87      5054

    accuracy                           0.89     11711
   macro avg       0.88      0.89      0.89     11711
weighted avg       0.89      0.89      0.89     11711



In [10]:
# Save to file in the current working directory
pkl_filename = "/home/jaarciah/PI/Clasificación /modelos/iteración_3/modelbaggingDTre.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(modelbaggingDTre, file)

### b. Nearest Neighbors.

In [11]:
modelbaggingNN = make_pipeline(BaggingClassifier(KNeighborsClassifier(algorithm='auto', leaf_size=30,metric='manhattan', metric_params=None,n_jobs=None, n_neighbors=25, p=2, weights='distance')))
modelbaggingNN.fit(X_train,
          y_train)
ypred = modelbaggingNN.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.76      0.74      0.75      6657
           1       0.67      0.68      0.68      5054

    accuracy                           0.72     11711
   macro avg       0.71      0.71      0.71     11711
weighted avg       0.72      0.72      0.72     11711



In [12]:
# Save to file in the current working directory
pkl_filename = "/home/jaarciah/PI/Clasificación /modelos/iteración_3/modelbaggingNN.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(modelbaggingNN, file)

### c. ExtraTree.

In [15]:
modelbaggingETre = make_pipeline(BaggingClassifier(ExtraTreeClassifier(ccp_alpha=0.0, class_weight=None,criterion='gini', max_depth=9,max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0,min_impurity_split=None,min_samples_leaf=4, min_samples_split=9,min_weight_fraction_leaf=0.0,random_state=None, splitter='best')))
modelbaggingETre.fit(X_train,
          y_train)
ypred = modelbaggingETre.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89      6657
           1       0.84      0.89      0.87      5054

    accuracy                           0.88     11711
   macro avg       0.88      0.88      0.88     11711
weighted avg       0.88      0.88      0.88     11711



In [16]:
# Save to file in the current working directory
pkl_filename = "/home/jaarciah/PI/Clasificación /modelos/iteración_3/modelbaggingETre.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(modelbaggingETre, file)